In [1]:
import numpy as np
from turtle import right
from torch.fft import ifft
from torch.fft import fft
from collections import Counter
import numpy as npstandardize
from utils import circular_correlation, load, Condition_Setter, circular_convolution, inverse_circular_correlation,set_random_seed
from sklearn.decomposition import PCA
import torch
import matplotlib.pyplot as plt
from torch.nn.functional import cosine_similarity as cos
from torch.nn.functional import normalize
import random
condition = Condition_Setter(set_embedding_type=False)
set_random_seed(4545)

Berkeley Parser初期化

In [2]:
import benepar, spacy
nlp = spacy.load('en_core_web_md')
if spacy.__version__.startswith('2'):
        nlp.add_pipe(benepar.BeneparComponent("benepar_en3_large"))
else:
    nlp.add_pipe("benepar", config={"model": "benepar_en3_large"})

In [13]:
print(len(tree_list))

1285


Hol-CCGでベクトルを合成

In [3]:
device = torch.device('cuda')
model = '/workspace/Hol-CCG/data/model/roberta-large_w_p_s_2023-01-20_11:57:41.pth'
tree_net = torch.load(model,map_location=device)
tree_net.device = device
dev_tree_list = load('/workspace/Hol-CCG/data/tree_list/dev_tree_list.pickle')
dev_tree_list.tokenizer = tree_net.tokenizer
dev_tree_list.set_info_for_training(tokenizer=tree_net.tokenizer)
with torch.no_grad():
    dev_tree_list.set_vector(tree_net)
# store vectors and contents for each node
vector_list = []
content_list = []
for tree in dev_tree_list.tree_list:
    for node in tree.node_list:
        if not node.is_leaf:
            vector_list.append(node.vector)
            content_list.append(node.content)
vector_list = torch.stack(vector_list)


setting vector...: 100%|██████████| 1913/1913 [00:36<00:00, 51.96it/s]


Decompositionし，類似する表現を検索

In [4]:
tree_list = []
for tree in dev_tree_list.tree_list:
    if len(tree.sentence) > 30 or len(tree.sentence) < 10:
        continue
    tree_list.append(tree)

# randomly sample 1 nodes from each tree whose content is longer than 2 and shorter than 6
for tree in tree_list:
    while True:
        node = random.choice(tree.node_list)
        if node.is_leaf or node.parent_node is None:
            continue
        elif len(node.content) < 2 or len(node.content) > 6:
            continue
        else:
            break
    tree.replace_target_node = node
    parent_node = tree.replace_target_node.parent_node
    if tree.node_list[parent_node.left_child_node_id] == tree.replace_target_node:
        tree.replace_target_node.lr = 'l'
        tree.sibling_node = tree.node_list[parent_node.right_child_node_id]
        reconstruct_vector = inverse_circular_correlation(parent_node.vector, 
                                                        tree.sibling_node.vector,
                                                        tree_net.vector_norm,
                                                        child_is_left=False)
        tree.replace_target_node.reconstruct_vector = reconstruct_vector
    else:
        tree.replace_target_node.lr = 'r'
        tree.sibling_node = tree.node_list[parent_node.left_child_node_id]
        reconstruct_vector = inverse_circular_correlation(parent_node.vector,
                                                        tree.sibling_node.vector,   
                                                        tree_net.vector_norm,
                                                        child_is_left=True)
        tree.replace_target_node.reconstruct_vector = reconstruct_vector
for tree in tree_list:
    vector = tree.replace_target_node.reconstruct_vector
    content = tree.replace_target_node.content
    # search top k similar vector from vector_list
    k = 1
    vector = vector.unsqueeze(0)
    similarity = cos(vector, vector_list)
    top_k = torch.topk(similarity, k+1)
    top_k_index = top_k.indices[1:]
    tree.replace_target_node.infilled_content = []
    for idx in top_k_index:
        tree.replace_target_node.infilled_content.append(content_list[idx])
    tree.original_sentence = ' '.join(tree.sentence)
    tree.infilled_sentences = []
    for content in tree.replace_target_node.infilled_content:
        infilled_sentence = tree.original_sentence.replace(' '.join(tree.replace_target_node.content), ' '.join(content))
        tree.infilled_sentences.append(infilled_sentence)

Berkeleyで解析

In [5]:
import tqdm
with tqdm.tqdm(total=len(tree_list)) as pbar:
    for tree in tree_list:
        doc = nlp(tree.original_sentence)
        sent = list(doc.sents)[0]
        tree.original_tree = sent
        tree.infilled_trees = []
        for infilled_sentence in tree.infilled_sentences:
            doc = nlp(infilled_sentence)
            sent = list(doc.sents)[0]
            tree.infilled_trees.append(sent)
        pbar.update(1)

  0%|          | 0/1285 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.8/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
100%|██████████| 1285/1285 [03:21<00:00,  6.38it/s]


置き換え対象の句の非終端記号を抽出

In [6]:
for tree in tree_list:
    target_phrase = ' '.join(tree.replace_target_node.content)
    tree.replace_target_node.original_symbol = None
    for constituent in tree.original_tree._.constituents:
        if constituent.text == target_phrase:
            tree.replace_target_node.original_symbol = str(constituent._.labels[0])
    tree.replace_target_node.infilled_symbols = []
    for target_phrase, infilled_tree in zip(tree.replace_target_node.infilled_content, tree.infilled_trees):
        target_phrase = ' '.join(target_phrase)
        infilled_symbol = None
        for constituent in infilled_tree._.constituents:
            if constituent.text == target_phrase:
                infilled_symbol = str(constituent._.labels[0])
                break
        tree.replace_target_node.infilled_symbols.append(infilled_symbol)

一致率を計算

In [7]:
num_match = 0
num_umatch = 0
for tree in tree_list:
    if tree.replace_target_node.original_symbol is None:
        continue
    else:
        for infilled_symbol in tree.replace_target_node.infilled_symbols:
            if infilled_symbol == tree.replace_target_node.original_symbol:
                num_match += 1
            else:
                num_umatch += 1
match_rate = num_match / (num_match + num_umatch)
print('num_match: {}'.format(num_match))
print('num_umatch: {}'.format(num_umatch))
print('match rate: {:.2f}%'.format(match_rate*100))

num_match: 808
num_umatch: 31
match rate: 96.31%


以下 RoBERTaの実験
RoBERTa MLMをLoad

In [8]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
device = 'cuda'
state_dict = torch.load('/workspace/Hol-CCG/data/model/roberta-mlm.pth')
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
model = RobertaForMaskedLM.from_pretrained('roberta-large')
model.load_state_dict(state_dict)
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNor

RoBERTaでInfilling

In [9]:
with tqdm.tqdm(total=len(tree_list)) as pbar:
    for tree in tree_list:
        original_sentence = tree.original_sentence
        target_phrase = ' '.join(tree.replace_target_node.content)
        tokenized_target_phrase = tokenizer.tokenize(target_phrase)
        mask_tokens = ''.join([tokenizer.mask_token] * len(tokenized_target_phrase))
        masked_sentence = original_sentence.replace(target_phrase, mask_tokens)
        input_ids = torch.tensor([tokenizer.encode(masked_sentence, add_special_tokens=True)]).to(device)
        predicted_tokens = []
        for i in range(len(tokenized_target_phrase)):
            with torch.no_grad():
                outputs = model(input_ids)
                logits = outputs.logits
            # get the index of the masked token
            masked_index = (input_ids == tokenizer.mask_token_id).nonzero()[0,1]
            # get the logits of the masked token
            logits = logits[0, masked_index,:]
            # get the top predicted token
            predicted_index = torch.argmax(logits).item()
            predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
            predicted_tokens.append(predicted_token)
            # replace input_ids with the predicted token
            input_ids[0, masked_index] = predicted_index
        tree.roberta_infilled_sentence = tokenizer.decode(input_ids[0,1:-1])
        infilled_phrase = tokenizer.convert_tokens_to_string(predicted_tokens).strip()
        tree.replace_target_node.roberta_infilled_phrase = infilled_phrase
        pbar.update(1)

100%|██████████| 1285/1285 [01:27<00:00, 14.63it/s]


Berkeleyで解析

In [10]:
import tqdm
with tqdm.tqdm(total=len(tree_list)) as pbar:
    for tree in tree_list:
        doc = nlp(tree.roberta_infilled_sentence)
        sent = list(doc.sents)[0]
        tree.roberta_infilled_tree = sent
        pbar.update(1)

100%|██████████| 1285/1285 [01:41<00:00, 12.70it/s]


置き換え対象の句の非終端記号を抽出

In [11]:
for tree in tree_list:
    target_phrase = tree.replace_target_node.roberta_infilled_phrase
    infilled_symbol = None
    for constituent in tree.roberta_infilled_tree._.constituents:
        if constituent.text == target_phrase:
            if len(constituent._.labels) > 0:
                infilled_symbol = str(constituent._.labels[0])
                break
    tree.replace_target_node.roberta_infilled_symbol = infilled_symbol

一致率を計算

In [12]:
num_match = 0
num_umatch = 0
for tree in tree_list:
    if tree.replace_target_node.original_symbol is None:
        continue
    else:
        if tree.replace_target_node.roberta_infilled_symbol == tree.replace_target_node.original_symbol:
            num_match += 1
        else:
            num_umatch += 1
match_rate = num_match / (num_match + num_umatch)

print('num_match: {}'.format(num_match))
print('num_umatch: {}'.format(num_umatch))
print('match rate: {:.2f}%'.format(match_rate*100))

num_match: 654
num_umatch: 185
match rate: 77.95%
